**Steam Table Simulation: Steam Compression Process**

In this tutorial, we will simulate a steam compression process using thermodynamic properties obtained from standard steam tables. This approach is commonly used for water and steam systems, where accurate property data (enthalpy, entropy, specific volume) is tabulated based on temperature and pressure.

We assume that the steam compressor operates at steady state.

### Inlet Specifications:

* **Mole Flow Rate**: 100 mol/s
* **Inlet Pressure**: 0.1 MPa
* **Inlet Temperature**: 373.15 K (100°C)

We will simulate two different cases based on compressor configuration inputs provided by the user:

---

### **Case 1: Fixed Isentropic Efficiency and Pressure Change**

In this scenario, we assume that the user specifies the desired pressure increase and the isentropic efficiency of the compressor. Steam properties at inlet and isentropic outlet conditions will be determined using steam tables.

* **Pressure Change**: 1.5 MPa
* **Isentropic Efficiency**: 0.85

---

### **Case 2: Fixed Isentropic Efficiency and Pressure Ratio**

Here, the user defines a pressure ratio instead of a pressure change. The final pressure is calculated from the inlet pressure and pressure ratio, and steam table data is used to compute the isentropic and actual outlet enthalpy.

* **Pressure Ratio**: 16
* **Isentropic Efficiency**: 0.85

---

For both cases, outlet enthalpies and power input to the compressor will be calculated using steam table data. This approach is ideal when working with saturated or superheated steam in power plant simulations or thermodynamic cycle analysis.

---


In [1]:
# Import pyomo package
from pyomo.environ import ConcreteModel, Constraint, value, units

# Import idaes logger to set output levels
import idaes.logger as idaeslog

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import the IAPWS property package to create a properties block for steam in the flowsheet
from idaes.models.properties import iapws95

from idaes.models.properties.iapws95 import htpx

# Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
from idaes.core.util.model_statistics import degrees_of_freedom

# Import the default IPOPT solver
from idaes.core.solvers import get_solver

# Import a compressor unit
from idaes.models.unit_models.pressure_changer import PressureChanger,ThermodynamicAssumption

# Import a feed and product stream
from idaes.models.unit_models import Feed, Product

## Setting up the flowsheet

Creating a `concretemodel` foundation, attach the steady state flowsheet and declaring the property package on the shell and tube side of the heat exchanger

In [2]:
m = ConcreteModel()

m.fs = FlowsheetBlock(dynamic=False)

m.fs.properties = iapws95.Iapws95ParameterBlock()

In [3]:
m.fs.steam_in_case_1 = Feed(property_package=m.fs.properties)
m.fs.steam_out_case_1 = Product(property_package=m.fs.properties)
m.fs.compressor_1 = PressureChanger(
    dynamic=False,
    property_package=m.fs.properties,
    compressor=True,
    thermodynamic_assumption=ThermodynamicAssumption.isentropic
)

# Call the degrees_of_freedom function, get initial DOF
DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))

The initial DOF is 5


## Connecting the unit operations

In [4]:
# Import necessary Pyomo components
from pyomo.environ import TransformationFactory

# Import the Arc component for connecting unit models
from pyomo.network import Arc

In [5]:
m.fs.s01 = Arc(source=m.fs.steam_in_case_1.outlet, destination = m.fs.compressor_1.inlet)
m.fs.s02 = Arc(source=m.fs.compressor_1.outlet, destination = m.fs.steam_out_case_1.inlet)

TransformationFactory("network.expand_arcs").apply_to(m)

# Call the degrees_of_freedom function, get initial DOF
degrees_of_freedom(m)

5

## Setting up the conditions

In [6]:
# Fix the stream inlet conditions
m.fs.steam_in_case_1.outlet.flow_mol[0].fix(100) # mol/s

# Use htpx method to obtain the molar enthalpy of inlet stream at the given temperature and pressure conditions 
m.fs.steam_in_case_1.outlet.enth_mol[0].fix(value(htpx(T=308.15*units.K, P=101325*units.Pa))) # T in K, P in Pa
m.fs.steam_in_case_1.outlet.pressure[0].fix(101325)

## Fixing the pressure change and isentropic efficiency

In [7]:
# Fix compressor conditions
m.fs.compressor_1.deltaP.fix(101325*5)
m.fs.compressor_1.efficiency_isentropic.fix(0.85)

# Call the degrees_of_freedom function, get final DOF
DOF_final = degrees_of_freedom(m)
print("The final DOF is {0}".format(DOF_final))

The final DOF is 0


## Initaialization

In [9]:
# Initialize the flowsheet, and set the output at INFO level
m.fs.compressor_1.initialize(outlvl=idaeslog.INFO)
m.fs.steam_in_case_1.initialize(outlvl=idaeslog.INFO)
m.fs.steam_out_case_1.initialize(outlvl=idaeslog.INFO)

2025-05-13 18:20:16 [INFO] idaes.init.fs.compressor_1: Initialization Complete: optimal - Optimal Solution Found
2025-05-13 18:20:16 [INFO] idaes.init.fs.steam_in_case_1: Initialization Complete.
2025-05-13 18:20:16 [INFO] idaes.init.fs.steam_out_case_1: Initialization Complete.


## Solve the model

In [10]:
# Solve the simulation using ipopt
# Note: If the degrees of freedom = 0, we have a square problem
from pyomo.environ import SolverFactory
opt = SolverFactory('ipopt')
solve_status = opt.solve(m, tee=True)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

In [11]:
# View results
m.fs.compressor_1.report()


Unit : fs.compressor_1                                                     Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key                   : Value      : Units         : Fixed : Bounds
    Isentropic Efficiency :    0.85000 : dimensionless :  True : (None, None)
          Mechanical Work :     1080.1 :          watt : False : (None, None)
          Pressure Change : 5.0662e+05 :        pascal :  True : (None, None)
           Pressure Ratio :     6.0000 : dimensionless : False : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                         Units           Inlet     Outlet  
    Molar Flow          mole / second     100.00     100.00
    Mass Flow       kilogram / second     1.8015     1.8015
    T                          kelvin     308.15     308.18
    P                          pascal 1.0132e+05 6.0795e+05
   

## Case 2 Fixed Isentropic Efficiency and Pressure Ratio

In [12]:
m.fs.steam_in_case_2 = Feed(property_package=m.fs.properties)
m.fs.steam_out_case_2 = Product(property_package=m.fs.properties)
m.fs.compressor_2 = PressureChanger(
    dynamic=False,
    property_package=m.fs.properties,
    compressor=True,
    thermodynamic_assumption=ThermodynamicAssumption.isentropic
)

# Call the degrees_of_freedom function, get initial DOF
DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))

m.fs.s03 = Arc(source=m.fs.steam_in_case_2.outlet, destination = m.fs.compressor_2.inlet)
m.fs.s04 = Arc(source=m.fs.compressor_2.outlet, destination = m.fs.steam_out_case_2.inlet)

TransformationFactory("network.expand_arcs").apply_to(m)

# Call the degrees_of_freedom function, get initial DOF
degrees_of_freedom(m)

# Fix the stream inlet conditions
m.fs.steam_in_case_2.outlet.flow_mol[0].fix(100) # mol/s

# Use htpx method to obtain the molar enthalpy of inlet stream at the given temperature and pressure conditions 
m.fs.steam_in_case_2.outlet.enth_mol[0].fix(value(htpx(T=308.15*units.K, P=101325*units.Pa))) # T in K, P in Pa
m.fs.steam_in_case_2.outlet.pressure[0].fix(101325)

# Fix compressor conditions
m.fs.compressor_2.ratioP.fix(16)
m.fs.compressor_2.efficiency_isentropic.fix(0.85)

# Call the degrees_of_freedom function, get final DOF
DOF_final = degrees_of_freedom(m)
print("The final DOF is {0}".format(DOF_final))

# Initialize the flowsheet, and set the output at INFO level
m.fs.compressor_2.initialize(outlvl=idaeslog.INFO)
m.fs.steam_in_case_2.initialize(outlvl=idaeslog.INFO)
m.fs.steam_out_case_2.initialize(outlvl=idaeslog.INFO)

opt = SolverFactory('ipopt')
solve_status = opt.solve(m, tee=True)

# View results
m.fs.compressor_2.report()

The initial DOF is 5
The final DOF is 0
2025-05-13 18:21:10 [INFO] idaes.init.fs.compressor_2: Initialization Complete: optimal - Optimal Solution Found
2025-05-13 18:21:10 [INFO] idaes.init.fs.steam_in_case_2: Initialization Complete.
2025-05-13 18:21:10 [INFO] idaes.init.fs.steam_out_case_2: Initialization Complete.
Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collect

In [13]:
m.fs.visualize("Compressor Models")

2025-05-13 18:21:15 [INFO] idaes.idaes_ui.fv.fsvis: Started visualization server
2025-05-13 18:21:15 [INFO] idaes.idaes_ui.fv.fsvis: Loading saved flowsheet from 'Compressor Models.json'
2025-05-13 18:21:15 [INFO] idaes.idaes_ui.fv.fsvis: Saving flowsheet to default file 'Compressor Models.json' in current directory (/home/viraj/Documents/Github/IDAES PSE Examples/Compressor)
2025-05-13 18:21:15 [WARNING] idaes.idaes_ui.fv.fsvis: Flowsheet name changed: old='Compressor Models' new='Compressor-Models'
Flowsheet name changed to 'Compressor-Models'
2025-05-13 18:21:16 [INFO] idaes.idaes_ui.fv.fsvis: Flowsheet visualization at: http://localhost:60323/app?id=Compressor-Models


VisualizeResult(store=<idaes_ui.fv.persist.FileDataStore object at 0x75d37623cd10>, port=60323, server=<idaes_ui.fv.model_server.FlowsheetServer object at 0x75d37822bbf0>, save_diagram=<bound method SaveDiagramScreenshot.save_diagram_screenshot of <idaes_ui.fv.save_diagram_screenshot.SaveDiagramScreenshot object at 0x75d376071400>>)